## Evaluating Classifiers

Today we are going to discuss metrics for comparing classifiers. We will bring back the rule-based classifier that was introduced yesterday (the classifier that used the word overlap proportion for headlines and articles in order to make its decision), and we will compare the performance of this classifier against a classifier that classifies every article-headline pair as 'unrelated'. We will see that accuracy is only one way to measure performance, and that it may not be the best metric when the training and test examples are unbalanced across classes.

### 1. Load the data 

We have copied the functions that we created yesterday into a separate file (named 'initial_classifier_utils.py'), and we will now import those functions here, along with the other python packages we need:

In [1]:
import initial_classifier_utils as utils
import numpy as np
import pandas as pd
import sys
import os.path
# Adjust settings so that we can fully see the dataset below
pd.set_option('display.max_colwidth', -1)

In [2]:
# Create the training data, if it doesn't already exist:
if os.path.isfile('train_data.csv') == False:
    utils.merge_data("train_stances.csv", "train_bodies.csv", "train_data.csv")

In [3]:
# Create the test data, if it doesn't already exist:
if os.path.isfile('test_data.csv') == False:
    utils.merge_data("competition_test_stances.csv", "competition_test_bodies.csv", "test_data.csv")

In [4]:
# Read in the training and test data and separate out the training data according to the value of the Stance variable:
train_data = pd.read_csv("train_data.csv", encoding = "utf-8")
unrelated_train = train_data[train_data['Stance'] == 'unrelated']
discuss_train = train_data[train_data['Stance'] == 'discuss']
agree_train = train_data[train_data['Stance'] == 'agree']
disagree_train = train_data[train_data['Stance'] == 'disagree']

test_data = pd.read_csv("test_data.csv", encoding = "utf-8")

### 2. Load the classifier and make predictions on the test data

In [5]:
# We are going to compute the headline-article word overlap for each of the examples in the unrelated,
# discuss, agree and disagree categories in the training set.  
# Warning: this may take a minute - it is iterating through almost 50,000 examples!
proportions_train = utils.compute_proportions(unrelated_train, discuss_train, agree_train, disagree_train)

In [6]:
# We are going to use a lot of the functions that were created yesterday, but we are also going to define a new one:
# The function below takes the test data set and iterates through each article-headline pair. For each article-headline pair
# it makes a prediction by calculating the headline-article word overlap and comparing this to the mean overlap values 
# for each of the categories in the training set 
def make_predictions(test_data, proportions_train):
    stance_counts = {"unrelated":0, "discuss":0, "agree":0, "disagree":0}
    stance_correct_counts = {"unrelated":0, "discuss":0, "agree":0, "disagree":0}
    predictions_list = []
    for i in range(test_data.shape[0]):
        example = test_data.iloc[i]
        predicted_stance = utils.make_prediction(example, proportions_train)
        actual_stance = example['Stance']
        # Append the predicted stance to the predictions_list - this will allow us to compare predictions and true 
        # values later on
        predictions_list.append(predicted_stance)
    return predictions_list

In [7]:
# Call the make_predictions function (it may take a minute, the test_data set contains 25,413 examples!) 
# and check that it is the right size (one way to check that the make_predictions function is performing as expected)
predictions = make_predictions(test_data, proportions_train)
len(predictions)

25413

### 3. Compare predictions and ground truth

In [8]:
# We are going to append the predictions list we just obtained to the test_data dataframe as an additional column,
# along with one more column that predicts 'unrelated' for every example:
test_data['prediction_1'] = predictions
test_data['prediction_2'] = 'unrelated'

In [9]:
# Let's look at the modified test_data dataframe by viewing the first few examples:
test_data.head()

,Body ID,articleBody,Headline,Stance,prediction_1,prediction_2
0,1,Al-Sisi has denied Israeli reports stating that he offered to extend the Gaza Strip.,Apple installing safes in-store to protect gold Watch Edition,unrelated,unrelated,unrelated
1,1,Al-Sisi has denied Israeli reports stating that he offered to extend the Gaza Strip.,El-Sisi denies claims he'll give Sinai land to Palestinians,agree,unrelated,unrelated
2,1,Al-Sisi has denied Israeli reports stating that he offered to extend the Gaza Strip.,Apple to keep gold Watch Editions in special in-store safes,unrelated,unrelated,unrelated
3,1,Al-Sisi has denied Israeli reports stating that he offered to extend the Gaza Strip.,Apple Stores to Keep Gold “Edition” Apple Watch in Custom Safes,unrelated,unrelated,unrelated
4,1,Al-Sisi has denied Israeli reports stating that he offered to extend the Gaza Strip.,South Korean woman's hair 'eaten' by robot vacuum cleaner as she slept,unrelated,unrelated,unrelated


In [10]:
# Let's look at a few more examples (this time at the end of the dataset):
test_data[-3:]

Body ID  \
25410  2586      
25411  2586      
25412  2586      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### 4. Compare classifier performance

We are now going to introduce some tools that we can use for evaluating classifier performance. These are:
- Accuracy 
- Confusion Matrices
- Precision and Recall
- F1 score

#### 4a. Accuracy

Accuracy is defined as the ratio of the number of examples that were correctly predicted compared to the number of examples in the test dataset:
$\text{accuracy} = \dfrac{\text{number correct}}{\text{number of examples}}$

Let's calculate accuracy values for the classifier we saw yesterday, and compare this to the classifier which predicts 'unrelated' for all examples:

In [11]:
def calculate_accuracy(test_data, truth_col_name, prediction_col_name):
    number_correct = sum(test_data[truth_col_name] == test_data[prediction_col_name])
    number_examples = test_data.shape[0]
    accuracy = number_correct/number_examples
    return accuracy

In [12]:
# Calculate accuracy for the classifier we created yesterday:
calculate_accuracy(test_data, 'Stance', 'prediction_1')

0.691575178058474

In [13]:
# Calculate accuracy for the classifier that classifies all examples as 'unrelated':
calculate_accuracy(test_data, 'Stance', 'prediction_2')

0.7220320308503522

#### 4b. Confusion Matrices

We have just seen that the classifier that predicts 'unrelated' for everything has a higher accuracy than the one we created yesterday! But there is something unsatisfying about this second classifier, and if we were using it to predict articles that might be Fake News (because the headline and article are unrelated), all articles would be marked as Fake News! This doesn't seem helpful! 

Perhaps accuracy is not the right tool to evaluate our classifier, and we should be using something else...

We are now going to calculate something called a 'confusion matrix' for each of our classifiers:

In [14]:
def calculate_confusion_matrix(test_data, truth_col_name, prediction_col_name):
    return pd.crosstab(test_data[truth_col_name], test_data[prediction_col_name])

In [15]:
calculate_confusion_matrix(test_data, 'Stance', 'prediction_1')

prediction_1,agree,disagree,discuss,related,unrelated
Stance,,,,,
agree,895,342,8,1,657
disagree,239,149,5,0,304
discuss,1977,862,16,1,1608
unrelated,211,1600,20,3,16515


What does the confusion matrix tell us? 

If we look at the first entry of the confusion matrix, with value 895, this tells us that 895 examples in the test dataset had label 'agree' (row value) and were correctly classified by our classifier as 'agree' (column value).

If we look at the entry in the second row and first column (value 239), this tells us that 239 examples in the test dataset had label 'disagree', but were classified as 'agree' by our classifier. 

In [16]:
# Let's also look at the confusion matrix, where the entries are given as a proportion of the size of the
#  test dataset
calculate_confusion_matrix(test_data, 'Stance', 'prediction_1')/test_data.shape[0]

prediction_1,agree,disagree,discuss,related,unrelated
Stance,,,,,
agree,0.035218,0.013458,0.000315,0.000039,0.025853
disagree,0.009405,0.005863,0.000197,0.000000,0.011962
discuss,0.077795,0.033920,0.000630,0.000039,0.063275
unrelated,0.008303,0.062960,0.000787,0.000118,0.649864


In [17]:
# Let's calculate the confusion matrix for the second classifier:
calculate_confusion_matrix(test_data, 'Stance', 'prediction_2')

prediction_2,unrelated
Stance,
agree,1903
disagree,697
discuss,4464
unrelated,18349


In [18]:
# And again, let's make the entries proportions of the full test dataset:
calculate_confusion_matrix(test_data, 'Stance', 'prediction_2')/test_data.shape[0]

prediction_2,unrelated
Stance,
agree,0.074883
disagree,0.027427
discuss,0.175658
unrelated,0.722032


TODO:
- Discussion of confusion matrices (and division by row percentage to look at accuracy for each category)
- F1 and precision/recall discussion.

### 5. Extra Challenge

1. Take a look at some of the examples that were misclassified by our proportion overlap classifier. Why do you think our proportion overlap model failed? Can you think of a rule that would properly classify these examples?  Why not test it out?